# Feature Extraction

In [1]:
import numpy as np
import pandas as pd
import sklearn
import pickle
import mne
import os
import matplotlib.pyplot as plt
from mne.time_frequency import tfr_multitaper

%matplotlib inline

# prevent extensive logging
mne.set_log_level('WARNING')

In [2]:
# Load the DataFrame from the pickle file
df_participants = pd.read_pickle('/content/drive/MyDrive/TD-BRAIN/TDBRAIN_participants_V2_data/df_participants.pkl')

# Drop the 'diagnosis_group' column
df_participants = df_participants.drop(columns=['diagnosis_group'])

# Print the updated DataFrame shape to confirm the column has been removed
print(f'all participants after removing diagnosis_group: {df_participants.shape}')

# Display a sample of the DataFrame to verify the change
df_participants.sample(5)


all participants after removing diagnosis_group: (370, 12)


participants_ID   DISC/REP indication formal_status   Dataset    age  \
408     sub-88010033  DISCOVERY        MDD           MDD  MDD-rTMS  30.64   
1086    sub-88058229  DISCOVERY        MDD           MDD  MDD-rTMS  49.94   
576     sub-88018937  DISCOVERY        MDD       UNKNOWN       NaN  45.97   
516     sub-88015249  DISCOVERY        MDD       UNKNOWN       NaN  27.94   
804     sub-88035501  DISCOVERY        MDD           MDD  MDD-rTMS  61.26   

      gender  sessID  nrSessions    EC    EO diagnosis  
408        0       1           1  True  True       MDD  
1086       1       1           1  True  True       MDD  
576        1       1           1  True  True       MDD  
516        0       1           1  True  True       MDD  
804        0       1           1  True  True       MDD

In [3]:
# Print de oorspronkelijke vorm en unieke ID's
print("Voor aanpassing:")
print("Vorm van de DataFrame:", df_participants.shape)
print("Unieke IDs:", df_participants['participants_ID'].nunique())


Voor aanpassing:
Vorm van de DataFrame: (370, 12)
Unieke IDs: 362


In [4]:
## Set montage based on channel names and locations provided in Van Dijk et al., (2022) (copied by Anne van Duijvenbode)

ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg',\
           'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', \
           'eog', 'eog', 'eog', 'eog', 'ecg', 'eog', 'emg']

ch_names = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC3', 'FCz', 'FC4', 'T7', 'C3', 'Cz', 'C4', 'T8', 'CP3', \
            'CPz', 'CP4', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'Oz', 'O2', 'VPVA', 'VNVB', 'HPHL', 'HNHR', 'Erbs', \
            'OrbOcc', 'Mass']

dict_eeg_channels =  {ch_names[i]: ch_types[i] for i in range(len(ch_types))}

dict_ch_pos = {'Fp1' : [-26.81, 84.06, -10.56],
               'Fp2' : [29.41, 83.74, -10.04],
               'F7'  : [-66.99, 41.69, -15.96],
               'F3'  : [-48.05, 51.87, 39.87],
               'Fz'  : [0.90, 57.01, 66.36],
               'F4'  : [50.38, 51.84, 41.33],
               'F8'  : [68.71, 41.16, -15.31],
               'FC3' : [-58.83, 21.02, 54.82],
               'FCz' : [0.57, 24.63, 87.63],
               'FC4' : [60.29, 21.16, 55.58],
               'T7'  : [-83.36, -16.52, -12.65],
               'C3'  : [-65.57, -13.25, 64.98],
               'Cz'  : [0.23, -11.28, 99.81],
               'C4'  : [66.50, -12.80, 65.11],
               'T8'  : [84.44, -16.65, -11.79],
               'CP3' : [-65.51, -48.48, 68.57],
               'CPz' : [-0.42, -48.77, 98.37],
               'CP4' : [65.03, -48.35, 68.57],
               'P7': [-71.46, -75.17, -3.70],
               'P3'  : [-55.07, -80.11, 59.44],
               'Pz'  : [-0.87, -82.23, 82.43],
               'P4'  : [53.51, -80.13, 59.40],
               'P8' : [71.10, -75.17, -3.69],
               'O1'  : [-28.98, -114.52, 9.67],
               'Oz'  : [-1.41, -117.79, 15.84],
               'O2'  : [26.89, -114.68, 9.45]
              }

dict_ch_pos_m = {'Fp1' : [-0.2681, 0.8406, -0.1056],
               'Fp2' : [0.2941, 0.8374, -0.1004],
               'F7'  : [-0.6699, 0.4169, -0.1596],
               'F3'  : [-0.4805, 0.5187, 0.3987],
               'Fz'  : [0.0090, 0.5701, 0.6636],
               'F4'  : [0.5038, 0.5184, 0.4133],
               'F8'  : [0.6871, 0.4116, -0.1531],
               'FC3' : [-0.5883, 0.2102, 0.5482],
               'FCz' : [0.0057, 0.2463, 0.8763],
               'FC4' : [0.6029, 0.2116, 0.5558],
               'T7'  : [-0.8336, -0.1652, -0.1265],
               'C3'  : [-0.6557, -0.1325, 0.6498],
               'Cz'  : [0.0023, -0.1128, 0.9981],
               'C4'  : [0.6650, -0.1280, 0.6511],
               'T8'  : [0.8444, -0.1665, -0.1179],
               'CP3' : [-0.6551, -0.4848, 0.6857],
               'CPz' : [-0.042, -0.4877, 0.9837],
               'CP4' : [0.6503, -0.4835, 0.6857],
               'P7'  : [-0.7146, -0.7517, -0.0370],
               'P3'  : [-0.5507, -0.8011, 0.5944],
               'Pz'  : [-0.0087, -0.8223, 0.8243],
               'P4'  : [0.5351, -0.8013, 0.5940],
               'P8'  : [0.7110, -0.7517, -0.0369],
               'O1'  : [-0.2898, -1.1452, 0.0967],
               'Oz'  : [-0.0141, -1.1779, 0.1584],
               'O2'  : [0.2689, -1.1468, 0.0945]
              }

dict_ch_pos_array = {'Fp1' : np.array([-0.02681, 0.08406, -0.01056]),
               'Fp2' : np.array([0.02941, 0.08374, -0.01004]),
               'F7'  : np.array([-0.06699, 0.04169, -0.01596]),
               'F3'  : np.array([-0.04805, 0.05187, 0.03987]),
               'Fz'  : np.array([0.00090, 0.05701, 0.06636]),
               'F4'  : np.array([0.05038, 0.05184, 0.04133]),
               'F8'  : np.array([0.06871, 0.04116, -0.01531]),
               'FC3' : np.array([-0.05883, 0.02102, 0.05482]),
               'FCz' : np.array([0.00057, 0.02463, 0.08763]),
               'FC4' : np.array([0.06029, 0.02116, 0.05558]),
               'T7'  : np.array([-0.08336, -0.01652, -0.01265]),
               'C3'  : np.array([-0.06557, -0.01325, 0.06498]),
               'Cz'  : np.array([0.000023, -0.01128, 0.09981]),
               'C4'  : np.array([0.06650, -0.01280, 0.06511]),
               'T8'  : np.array([0.08444, -0.01665, -0.01179]),
               'CP3' : np.array([-0.06551, -0.04848, 0.06857]),
               'CPz' : np.array([-0.0042, -0.04877, 0.09837]),
               'CP4' : np.array([0.06503, -0.04835, 0.06857]),
               'P7'  : np.array([-0.07146, -0.07517, -0.00370]),
               'P3'  : np.array([-0.05507, -0.08011, 0.05944]),
               'Pz'  : np.array([-0.00087, -0.08223, 0.08243]),
               'P4'  : np.array([0.05351, -0.08013, 0.05940]),
               'P8'  : np.array([0.07110, -0.07517, -0.00369]),
               'O1'  : np.array([-0.02898, -0.11452, 0.00967]),
               'Oz'  : np.array([-0.00141, -0.11779, 0.01584]),
               'O2'  : np.array([0.02689, -0.11468, 0.00945])
              }

# channel groupings
frontal = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC3', 'FCz', 'FC4']
central = ['T7', 'C3', 'Cz', 'C4', 'T8']
parietal = ['CP3','CPz', 'CP4', 'P7', 'P3', 'Pz', 'P4', 'P8']
occipital = ['O1', 'Oz', 'O2']
channel_groups = {'frontal': frontal, 'central': central, 'parietal': parietal, 'occipital': occipital}

# define (5) frequencies of interest for TFR per frequency band
delta = np.array([1, 1.5, 2, 2.5, 3]) # starting at one because of high-pass filter
theta = np.array([4, 4.75, 5.5, 6.25, 7])
alpha = np.array([8, 9, 10, 11, 12])
beta = np.array([13, 17.25, 21.5, 25.75, 30])
gamma = np.array([42, 54, 66, 78, 90])
bands = {'delta': delta, 'theta': theta, 'alpha': alpha, 'beta': beta, 'gamma': gamma}


## Create montage
montage = mne.channels.make_dig_montage(ch_pos = dict_ch_pos_array, coord_frame = 'head')

# Create info object for MNE
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=500)
info.set_montage(montage=montage, on_missing= 'raise')
print(info)

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F7, F3, Fz, F4, F8, FC3, FCz, FC4, T7, C3, Cz, C4, T8, ...
 chs: 26 EEG, 5 EOG, 1 ECG, 1 EMG
 custom_ref_applied: False
 dig: 29 items (3 Cardinal, 26 EEG)
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 500.0 Hz
>


# Feature Extraction and storing in df

In this section of the study, I focus on extracting relevant features from EEG data to analyze Frontal Alpha Asymmetry (FAA) as a biomarker for psychiatric conditions. The EEG data, specifically for Eyes Closed (EC) and Eyes Open (EO) conditions, were recorded in a single session and preprocessed using the MNE software (version 1.2.1). The preprocessing steps included setting the electrode montage manually based on specified coordinates, applying a band-pass filter (0.5 Hz to 100 Hz), and conducting Independent Component Analysis (ICA) to remove artifacts associated with eye movements, cardiac signals, and muscle contractions. The efficacy of ICA was visually confirmed by inspecting components for selected participant-condition combinations.

Post-preprocessing, the EEG data were segmented into fixed-length epochs of 9.95 seconds with no overlap between them. Each epoch was then subjected to a Power Spectral Density (PSD) analysis using the Welch method, focusing specifically on the alpha band (8-13 Hz). This analysis was confined to the frontal EEG channels (Fp1, Fp2, F7, F8, F3, F4, FC3, and FC4), as these locations are crucial for computing FAA.

Frontal Alpha Asymmetry was calculated by deriving the difference in mean alpha power between pairs of symmetrical frontal electrodes. The FAA values were calculated for each epoch and then integrated into participant-specific feature datasets for both EC and EO conditions. To enhance the model’s sensitivity and ensure numerical stability, the power values were scaled by
10^12, converting them into a range more suitable for subsequent deep learning analysis.

These extracted features, representing variations in frontal brain activity under different conditions, were then used to train models to identify and predict psychiatric diagnoses, leveraging the potential of EEG as a non-invasive diagnostic tool. The process of feature extraction not only underscores the technical depth of handling EEG data but also enhances the reliability of interpreting EEG as a biomarker in clinical and research settings.









In [5]:
import scipy
# calculate variance in power per freq band and per channel group for each file and store in dataframe
eeg_dir = "/content/drive/MyDrive/TD-BRAIN/preprocessed"

sample_ids = df_participants['participants_ID'].tolist() # list of participants to include

df_ec_features = pd.DataFrame() # create empty dataframe to store EC features
df_eo_features = pd.DataFrame() # create empty dataframe to store EO features

# counter for progress
count = 1
if count == 1:
    total_files = 0
    for _, dirs, files in os.walk(eeg_dir):
        #dirs[:] = [d for d in dirs if d not in exlude_dirs] # exclude directories
        total_files += len([file for file in files if any(sample_id in file for sample_id in sample_ids) and '.npy' in file and 'ses-1' in file and 'BAD' not in file])

for subdir, dirs, files in os.walk(eeg_dir): # iterate through all files
    #dirs[:] = [d for d in dirs if d not in exlude_dirs] # exclude directories
    for file in files:
        if any(sample_id in file for sample_id in sample_ids): # filter participants to include
            if 'ses-1' in file and '.npy' in file and 'BAD' not in file: # filter first session, .npy files, and non-bad files
                filepath = os.path.join(subdir, file) # path to eeg file

                # needs specific info object, because has one less channel
                info = mne.create_info(ch_names=ch_names[:32], ch_types=ch_types[:32], sfreq=500)
                info.set_montage(montage=montage, on_missing= 'raise')

                preprocessed_eeg = np.load(filepath, allow_pickle = True)
                raw = mne.io.RawArray(np.squeeze(preprocessed_eeg['data']), info)

                # epoch the data
                epochs = mne.make_fixed_length_epochs(raw, duration = 9.95, overlap = 0)

                if 'EC' in file:
                    cond = 'EC'
                if 'EO' in file:
                    cond = 'EO'

                # determine age, gender, and diagnosis of participant
                age = df_participants.loc[df_participants['participants_ID'] == file.split('_')[0], 'age'].values[0]
                gender = df_participants.loc[df_participants['participants_ID'] == file.split('_')[0], 'gender'].values[0]
                diagnosis = df_participants.loc[df_participants['participants_ID'] == file.split('_')[0], 'diagnosis'].values[0]


                # add data to empty dictionary
                feature_dict = {}
                feature_dict['ID'] = [file.split('_')[0]] * epochs.get_data().shape[0]
                feature_dict['age'] = [age] * epochs.get_data().shape[0]
                feature_dict['gender'] = [gender] * epochs.get_data().shape[0]
                feature_dict['diagnosis'] = [diagnosis] * epochs.get_data().shape[0]
                #feature_dict['EO/EC'] = [cond] * epochs.get_data().shape[0]
                feature_dict['epoch'] = list(range(1, epochs.get_data().shape[0] + 1))

                # calculate psd for each epoch of frontal channels
                spectrum = epochs.compute_psd(method='welch', fmin=8, fmax=13, picks=['Fp1', 'Fp2', 'F7', 'F8', 'F3', 'F4', 'FC3', 'FC4'])
                psds, freqs = spectrum.get_data(return_freqs=True)

                # Compute mean alpha power per channel
                psd_alpha = np.mean(psds, axis=2)
                # print(f'{psds.shape = }')
                # print(f'{psd_alpha.shape = }')
                # Creating dataframe
                df = pd.DataFrame(data=psd_alpha,
                                  columns=['Fp1', 'Fp2', 'F7', 'F8', 'F3', 'F4', 'FC3', 'FC4'],
                                  index=list(range(1, epochs.get_data().shape[0] + 1))
                                  )
                # print(df)

                # calculate frontal alpha asymmetry
                asymmetry = pd.DataFrame(columns=['Fp2-Fp1', 'F8-F7', 'F4-F3', 'FC4-FC3'], index=list(range(1, epochs.get_data().shape[0] + 1)))
                even_columns = ["Fp2", "F8", "F4", "FC4"]
                uneven_columns = ["Fp1", "F7", "F3", "FC3"]
                asymmetry[["Fp2-Fp1", "F8-F7", "F4-F3", "FC4-FC3"]] = (df[even_columns] - df[uneven_columns].values) * 1e12
                asymmetry.index.name='Epoch'
                # print(asymmetry)

                # Convert asymmetry DataFrame to dictionary
                asymmetry_dict = asymmetry.to_dict(orient='list')
                # print(asymmetry_dict)

                if cond == 'EC':
                    asymmetry_dict = {key + '_ec': value for key, value in asymmetry_dict.items()}
                if cond == 'EO':
                    asymmetry_dict = {key + '_eo': value for key, value in asymmetry_dict.items()}

                # Merge asymmetry_dict into feature_dict
                feature_dict.update(asymmetry_dict)
                # print(feature_dict)

                # add to dataframe
                if cond == 'EC':
                    df_ec_features = pd.concat([df_ec_features, pd.DataFrame(feature_dict)], ignore_index = True)
                if cond == 'EO':
                    df_eo_features = pd.concat([df_eo_features, pd.DataFrame(feature_dict)], ignore_index = True)

                print(f'\rProgress: {count}/{total_files} files processed.', end = '')
                count += 1

# merge EO and EC dataframes
df_features = pd.merge(df_eo_features, df_ec_features.drop(columns=['age', 'gender', 'diagnosis']),  how='outer', on=['ID', 'epoch'])
del df_ec_features, df_eo_features # remove dataframes to free up memory
print(f'\n{df_features.shape = }')
df_features.sample(12)

Progress: 713/713 files processed.
df_features.shape = (4308, 13)


ID    age  gender diagnosis  epoch    Fp2-Fp1_eo  \
878   sub-88017821  49.22     1.0       MDD      3  4.331141e+11   
2619  sub-88048325  22.29     1.0       MDD      4 -1.148053e+10   
1617  sub-88029689  38.38     0.0       MDD     10  1.395767e+11   
2738  sub-88049813  47.01     0.0       MDD      3  3.158283e+11   
2876  sub-88052329  52.99     0.0       MDD      9  4.489517e+11   
2018  sub-88038069  54.21     1.0       MDD      3  6.337899e+10   
1674  sub-88030281  51.28     1.0       MDD      7  9.617483e+10   
1742  sub-88032665  40.96     1.0       MDD      3  1.608316e+11   
1744  sub-88032665  40.96     1.0       MDD      5 -1.265268e+11   
3363  sub-88062409  28.09     0.0       MDD      4 -3.175069e+11   
147   sub-87976641    NaN     0.0   HEALTHY      4  2.155879e+11   
1870  sub-88034645  49.14     0.0       MDD     11  5.274556e+11   

          F8-F7_eo      F4-F3_eo    FC4-FC3_eo    Fp2-Fp1_ec      F8-F7_ec  \
878  -9.031669e+10  7.936286e+11  4.360395e+11  3.276517e+11  1.547361e+11   
2619  8.671774e+11  4.147960e+11  1.788855e+12  1.118586e+12  6.962057e+11   
1617 -2.955519e+11  1.559765e+11  1.519909e+09 -6.914785e+10 -4.776929e+11   
2738  3.059580e+11  6.434937e+11  6.167248e+11 -1.818213e+11 -1.804273e+11   
2876  2.067755e+11  7.193554e+09  1.464463e+12 -2.642328e+11 -6.240977e+11   
2018 -3.644373e+11 -1.799243e+11 -1.531092e+10  4.763850e+11 -3.419172e+11   
1674  5.753907e+10 -1.311934e+11 -1.881353e+10  4.258721e+09 -3.558974e+11   
1742  3.741156e+11  3.013004e+11  3.823784e+11  1.070432e+12  2.746395e+12   
1744  5.013157e+10  1.053173e+11  8.369345e+10  6.365072e+09  7.326102e+10   
3363  1.464447e+11  9.068951e+10 -5.020890e+10 -2.895801e+11 -1.073544e+12   
147   7.206904e+10  8.151557e+11  1.595881e+12  3.795421e+12  3.565635e+12   
1870 -6.524223e+11 -1.285648e+12 -1.826101e+12  4.249824e+11  5.291432e+11   

          F4-F3_ec    FC4-FC3_ec  
878   5.785553e+11 -3.035874e+11  
2619 -6.302070e+11  1.124089e+12  
1617  1.458920e+11 -3.148950e+10  
2738 -7.432132e+10 -2.988003e+11  
2876  3.150396e+10  6.373875e+10  
2018  1.295533e+12  4.670850e+11  
1674  6.689320e+11  5.587584e+11  
1742  1.861385e+12  1.661382e+12  
1744  9.905598e+10  8.434024e+10  
3363 -4.343376e+11 -9.755323e+11  
147  -2.163098e+12  1.700736e+12  
1870  3.145747e+11 -7.346318e+11

In [6]:
ec = df_features.loc[:, df_features.columns.str.contains('ec')]
df_eo = df_features.loc[:, df_features.columns.str.contains('eo')]

In [7]:
df_features.sample(3)

ID    age  gender diagnosis  epoch    Fp2-Fp1_eo  \
3796  sub-88069737  49.07     0.0       MDD      5  2.829866e+11   
2097  sub-88039773  24.18     1.0       MDD     10  1.184434e+11   
3062  sub-88055749  44.50     1.0       MDD      3 -1.638298e+11   

          F8-F7_eo      F4-F3_eo    FC4-FC3_eo    Fp2-Fp1_ec      F8-F7_ec  \
3796  4.404799e+11 -7.810989e+11 -1.607496e+12 -6.335933e+11 -3.462557e+12   
2097 -1.002075e+11 -1.359774e+10  1.801314e+11 -1.049875e+11 -4.453214e+11   
3062  5.689091e+10  1.843054e+11  1.291741e+12  6.274617e+11 -2.429006e+10   

          F4-F3_ec    FC4-FC3_ec  
3796 -2.024282e+12 -2.630936e+12  
2097 -3.760677e+11 -1.002125e+12  
3062  2.182783e+12  5.029530e+12

In [8]:
df_features.isna().sum() # check for missing values -> decent amount of missing EC data (84 entries = ~7 participants)

ID              0
age           132
gender         12
diagnosis      12
epoch           0
Fp2-Fp1_eo     12
F8-F7_eo       12
F4-F3_eo       12
FC4-FC3_eo     12
Fp2-Fp1_ec     48
F8-F7_ec       48
F4-F3_ec       48
FC4-FC3_ec     48
dtype: int64

In [9]:
missing_diagnosis = df_features[df_features['diagnosis'].isna()]
missing_diagnosis_ids = missing_diagnosis['ID'].unique()
print(missing_diagnosis_ids)


['sub-88030193']


In [10]:
df_features_2 = df_features


In [11]:
df_features_2.to_pickle('/content/drive/MyDrive/TD-BRAIN/extracted_features/df_stat_features.pkl')